In [32]:
import os
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
import pandas as pd
import altair
import seaborn
import folium
from folium import plugins
from IPython.display import display
RANDOM_SEED = 511
rng = np.random.default_rng(RANDOM_SEED)
seaborn.set_theme(style="darkgrid")
altair.renderers.enable('mimetype')
db_url = os.getenv('DB_URL')
%load_ext autoreload
%autoreload 2
import geopandas
import geoplot

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 1. QAing District Geometries
* us congress
* ny state senate
* ny state assembly
* nyc city council

and also:
* nyc cd (maybe)
* nyc zip code

Note: districts boundaries mostly include water areas but zip_codes don't. 
Community districts ARE so we can't do any fractional-part things accurately.

In [33]:
sql = """
SELECT 
    office,
    district,
    name,
    party,
    boundary
FROM representatives
WHERE office =
    --'House'
    --'State Senate'
    --'State Assembly'
   -- 'City Council'
"""

sql = """
    select 
        zip_code, boundary, neighborhood_name
    from zip_codes
    where boundary is not null
"""

sql = """
    select 
        borough_district_code, boundary, community_board_name
    from community_districts
    where boundary is not null
"""

df = geopandas.GeoDataFrame.from_postgis(sql, db_url, geom_col='boundary')
print(df.shape)
print(df.isnull().sum())
display(df.head(3).T)

df.explore(
    legend=True,
    style_kwds={'stroke': False}
)

(59, 3)
borough_district_code    0
boundary                 0
community_board_name     0
dtype: int64


,0,1,2
borough_district_code,404,304,303
boundary,MULTIPOLYGON Z (((-73.8850919561427 40.7283602...,MULTIPOLYGON Z (((-73.89738381304649 40.680168...,MULTIPOLYGON Z (((-73.9164574753002 40.6767260...
community_board_name,"Corona, Corona Heights, Elmhurst, and Newtown",Bushwick,"Bedford-Stuyvesant, Stuyvesant Heights, and Oc..."


# 2. Aggregate Statistics by District



For each representative (council, senate, assembly, house):
* for their district
+ for Harlem (or East Harlem alone)
+ for each borough
+ for NYC as a whole

* Population + % of total
* OTP patients admitted + % of total + % of population + rank in city
* OTP admissions + % of total + % of population + rank in city
    * need to use admissions to compare to patients
* OTP patients / OTP admissions
* OTP enrollment (or capacity) + % of total + % of population + rank in city
* Mean travel time and fraction
* ODs + % of total + % of population + rank in city?

* drug arrests?
* felony arrests?

since ODs are by UHF and a lot of these are by zip code
we have to use overlaps with the districts... let's also measure:
* (district area) / (union of district area with everything it intersects with)
to see if we should throw some of these statistics out.
